In [1]:
import os
import itertools
from benchmarkrewriter.benchmark_parser import BenchmarkParser, WorkerBenchmarkParser, WorkerEncoding
import json
import pandas as pd
import math

In [8]:
benchmark_path = '/home/dhu/Downloads/benchmarks_with_workers'
results_path = '/home/dhu/Downloads/results/'
ga_path = '/home/dhu/Downloads/converted/'

In [3]:
def worker_flexibility(benchmark : WorkerEncoding):
    n_assignments = 0
    m = benchmark.n_machines()
    o = benchmark.n_operations()
    durations = benchmark.durations()
    w = durations.shape[2]
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    n_assignments += 1
    average_assignments = n_assignments / o
    return average_assignments / n_assignments#(m*w)

def get_flexibility_and_dv(benchmark):
    all = 0
    unique = []
    durations = benchmark.durations()
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            if durations[i][j] > 0:
                if durations[i][j] not in unique:
                    unique.append(durations[i][j])
                all += 1.0

    return (all / len(durations)) / benchmark.n_machines(), len(unique) / all

def get_flexibility_and_dv_worker(benchmark):
    all = 0
    unique = []
    machines_available = 0
    durations = benchmark.durations()
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    if durations[i][j][k] not in unique:
                        unique.append(durations[i][j][k])
                    all += 1
            if any([x > 0 for x in durations[i][j]]):
                machines_available+=1
    #return (machines_available / len(durations)) / benchmark.n_machines(), len(unique) / all
    return worker_flexibility(benchmark), len(unique) / all
                

In [4]:
def read_benchmarks(path):
    sources = os.listdir(path)
    result = dict()
    for source in sources:
        b_path = path + '\\' + source + '\\'
        benchmarks = os.listdir(b_path)
        for benchmark in benchmarks:
            parser = BenchmarkParser()
            #parser = WorkerBenchmarkParser()
            data = parser.parse_benchmark(b_path + '\\' + benchmark)
            f, dv = get_flexibility_and_dv(data)
            #f, dv = get_flexibility_and_dv_worker(data)
            result[benchmark[:-4]] = {'n_operations': data.n_operations(), 'flexibility': f, 'duration_variety': dv, 'n_machines': data.n_machines()}

    return result

def get_max(durations):
    max = 0
    for operation in durations:
        for machine in operation:
            for worker in machine:
                if worker > max:
                    max = worker
    return max

def read_benchmarks_workers(path):
    result = dict()
    benchmarks = os.listdir(path)
    for benchmark in benchmarks:
        #parser = BenchmarkParser()
        parser = WorkerBenchmarkParser()
        data = parser.parse_benchmark(path + '/' + benchmark)
        #f, dv = get_flexibility_and_dv(data)
        f, dv = get_flexibility_and_dv_worker(data)
        instance_name = remap(benchmark[2:-12])
        metrics = dict()

        max_duration = get_max(data.durations())
        counts = [0] * (max_duration+1)
        for operation in data.durations():
            for machine in operation:
                for worker in machine:
                    if worker > 0:
                        counts[worker] += 1
        d_distinct = [x for x in range(len(counts)) if counts[x] > 0]
        d_unique = [x for x in range(len(counts)) if counts[x] == 1]
        d_shared = [x for x in range(len(counts)) if counts[x] > 1]
        metrics['d_distinct'] = d_distinct
        metrics['d_unique'] = d_unique
        metrics['d_shared'] = d_shared
        metrics['d_average'] = sum(counts)/data.n_operations()
        result[instance_name] = {'n_operations': data.n_operations(), 'flexibility': f, 'duration_variety': dv, 'n_machines': data.n_machines(), 'additional_metrics': metrics}

    return result

def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[0].startswith('Behnke'):
        return 'Behnke'+values[1]
    if values[0].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[1]
    if values[0].startswith('Chambers'):
        return 'ChambersBarnes'+values[1]
    if values[0].startswith('HurinkS'):
        return 'HurinkSdata'+values[1]
    if values[0].startswith('HurinkE'):
        return 'HurinkEdata'+values[1]
    if values[0].startswith('HurinkR'):
        return 'HurinkRdata'+values[1]
    if values[0].startswith('HurinkV'):
        return 'HurinkVdata'+values[1]
    if values[0].startswith('DP'):
        return 'DPpaulli'+values[1]
    if values[0].startswith('Kacem'):
        return 'Kacem'+values[1]
    if values[0].startswith('Fattahi'):
        return 'Fattahi'+values[1]
    return name

def read_results(path):
    files = os.listdir(path)
    #files = [path_to_results + r'results_ortools.txt', path_to_results +r'results_cplex_cp.txt', path_to_results +r'results_gurobi.txt', path_to_results +r'results_hexaly_rewritten.txt', path_to_results +r'results_cplex_lp_rewritten.txt']
    all_data = dict()
    data_as_dict = dict()
    known_optima = dict()
    for file in files:
        statuses = []
        optimal = 0
        feasible = 0
        infeasible = 0
        file_content = pd.read_csv(path + file, names=['name','optimization_status','fitness_value','lower_bound', 'runtime', 'result_vector1', 'result_vector2', 'result_vector3', 'peak_cpu', 'peak_ram', 'resource_history', 'best_result_history'], sep=';')#, converters={'best_result_history': pd.eval})
        df = pd.DataFrame(file_content)
        name = file.split('\\')[-1][8:-4]
        if name == 'hexaly_rewritten':
            name = 'hexaly'#all_data['results_hexaly'] = df
        elif name == 'cplex_lp_rewritten':
            name = 'cplex_lp'#all_data['results_cplex_lp'] = df
        all_data[name] = df
        for index, row in df.iterrows():
            if row['name'].startswith('Error'):
                infeasible += 1
                continue
            if name not in data_as_dict:
                data_as_dict[name] = dict()
            instance_name = remap(row['name'][2:-12])
            if name == 'hexaly' or not row['optimization_status'] < 0:
                data_as_dict[name][instance_name] = row['fitness_value']
                if row['optimization_status'] == 1.0:
                    found = False
                    for optima in known_optima:
                        if optima[0] == instance_name:
                            found = True
                            break
                    if not found:
                        known_optima[instance_name] = row['fitness_value']#.append((instance_name, row['fitness_value']))
                    optimal += 1
                else:
                    feasible += 1
            else:
                infeasible += 1
            if row['optimization_status'] not in statuses:
                statuses.append(row['optimization_status'])
        print(f'{name}: {statuses} - optimal: {optimal}, feasible: {feasible}, infeasible: {infeasible}')
    return data_as_dict, known_optima

def read_ga_results(path):
    results = dict()
    files = os.listdir(path)
    results['ga_best'] = dict()
    results['ga_average'] = dict()
    for file in files:
        if file == 'converted':
            continue
        with open(path + file, 'r') as f:
            ga_data = json.loads(f.read())

            
            for key in ga_data.keys():
                results['ga_best'][remap(key)] = ga_data[key]['best']
                results['ga_average'][remap(key)] = ga_data[key]['average']
    return results


In [6]:
results_data, known_optima = read_results(results_path)

gurobi: [0.0, -1.0, 1.0] - optimal: 15, feasible: 305, infeasible: 70
cplex_lp: [1.0, 0.0, -1.0] - optimal: 12, feasible: 309, infeasible: 81
hexaly: [-1] - optimal: 0, feasible: 402, infeasible: 0
hexaly: [-1, 1] - optimal: 29, feasible: 373, infeasible: 0
cplex_cp: [0.0, 1.0] - optimal: 55, feasible: 345, infeasible: 2
ortools: [0.0, 1.0] - optimal: 39, feasible: 358, infeasible: 5
cplex_lp: [0.0, -1.0] - optimal: 0, feasible: 321, infeasible: 81


In [9]:
ga_data = read_ga_results(ga_path)
for key in ga_data:
    results_data[key] = ga_data[key]
    optimal = 0
    feasible = 0
    infeasible = 0 # will stay 0, obviously
    for instance in ga_data[key]:
        if instance in known_optima and ga_data[key][instance] == known_optima[instance]:
            optimal += 1
        else:
            feasible += 1
#print(f'{key}:[0.0, 1.0] - optimal: {optimal}, feasible: {feasible}, infeasible: {infeasible}')
print(len(known_optima))
print(known_optima)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
benchmark_data = read_benchmarks_workers(benchmark_path)